# 13. Обнаружение и отслеживание объектов


## Вычисление разности между кадрами

In [2]:
import cv2
import numpy as np
import os

def frame_diff(prev_frame, cur_frame, next_frame):
    """Вычисляет разность между кадрами для обнаружения движения."""
    diff_frames_1 = cv2.absdiff(next_frame, cur_frame)
    diff_frames_2 = cv2.absdiff(cur_frame, prev_frame)
    result = cv2.bitwise_and(diff_frames_1, diff_frames_2)
    
    # Пороговая обработка
    _, thresh = cv2.threshold(result, 30, 255, cv2.THRESH_BINARY)
    
    # Морфологические операции
    kernel = np.ones((3, 3), np.uint8)
    thresh = cv2.dilate(thresh, kernel, iterations=2)
    
    return thresh


def process_frame(frame, scaling_factor):
    """Обрабатывает кадр: изменяет размер и преобразует в серый."""
    frame = cv2.resize(frame, None, fx=scaling_factor,
                      fy=scaling_factor, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)
    return gray


def create_synthetic_video(filename='test_video.avi', duration=10):
    """Создает синтетическое видео с движущимся объектом."""
    print(f"Создание тестового видео '{filename}'...")
    
    width, height = 640, 480
    fps = 20
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(filename, fourcc, fps, (width, height))
    
    frames = duration * fps
    
    for i in range(frames):
        # Создаем черный фон
        frame = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Движущийся круг
        x = int((width - 100) * (i / frames) + 50)
        y = height // 2 + int(50 * np.sin(i * 0.1))
        cv2.circle(frame, (x, y), 30, (255, 255, 255), -1)
        
        # Движущийся прямоугольник
        rect_x = int((width - 150) * ((frames - i) / frames) + 75)
        rect_y = height // 3
        cv2.rectangle(frame, (rect_x, rect_y), (rect_x + 60, rect_y + 60), 
                     (200, 200, 200), -1)
        
        # Случайный шум (имитация движения)
        if i % 5 == 0:
            noise_x = np.random.randint(0, width - 50)
            noise_y = np.random.randint(0, height - 50)
            cv2.circle(frame, (noise_x, noise_y), 15, (150, 150, 150), -1)
        
        out.write(frame)
    
    out.release()
    print(f"✓ Тестовое видео создано: {filename}")
    return filename


def find_camera():
    """Ищет доступную камеру."""
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            ret, frame = cap.read()
            cap.release()
            if ret and frame is not None:
                return i
    return None


def get_video_source():
    """Определяет источник видео: камера, файл или синтетическое видео."""
    print("=" * 50)
    print("СИСТЕМА ОБНАРУЖЕНИЯ ДВИЖЕНИЯ")
    print("=" * 50)
    
    # Пытаемся найти камеру
    print("\n1. Поиск веб-камеры...")
    camera_index = find_camera()
    
    if camera_index is not None:
        print(f"✓ Найдена камера с индексом {camera_index}")
        use_camera = input("Использовать камеру? (y/n): ").lower()
        if use_camera == 'y':
            return cv2.VideoCapture(camera_index), "Камера"
    
    # Проверяем наличие видеофайла
    print("\n2. Поиск видеофайлов...")
    video_files = [f for f in os.listdir('.') if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    
    if video_files:
        print("Найдены видеофайлы:")
        for idx, vf in enumerate(video_files):
            print(f"  [{idx}] {vf}")
        
        choice = input("Введите номер файла или 'n' для создания тестового видео: ")
        if choice.isdigit() and 0 <= int(choice) < len(video_files):
            filename = video_files[int(choice)]
            return cv2.VideoCapture(filename), f"Файл: {filename}"
    
    # Создаем синтетическое видео
    print("\n3. Создание тестового видео...")
    test_video = create_synthetic_video()
    return cv2.VideoCapture(test_video), f"Тестовое видео: {test_video}"


def main():
    """Основная функция программы."""
    
    # Получаем источник видео
    cap, source_name = get_video_source()
    
    if not cap.isOpened():
        print("❌ Ошибка: не удалось открыть источник видео")
        return
    
    print(f"\n{'=' * 50}")
    print(f"Источник: {source_name}")
    print(f"{'=' * 50}")
    print("\nНажмите ESC для выхода")
    print("Нажмите SPACE для паузы\n")
    
    scaling_factor = 0.5
    paused = False
    
    # Захват первых трёх кадров
    ret1, frame1 = cap.read()
    ret2, frame2 = cap.read()
    ret3, frame3 = cap.read()
    
    if not (ret1 and ret2 and ret3):
        print("❌ Ошибка чтения кадров")
        cap.release()
        return
    
    prev_frame = process_frame(frame1, scaling_factor)
    cur_frame = process_frame(frame2, scaling_factor)
    next_frame = process_frame(frame3, scaling_factor)
    
    frame_count = 3
    
    # Основной цикл
    while True:
        if not paused:
            # Обнаружение движения
            motion = frame_diff(prev_frame, cur_frame, next_frame)
            
            # Создаем цветную визуализацию
            motion_colored = cv2.applyColorMap(motion, cv2.COLORMAP_JET)
            
            # Обновление кадров
            prev_frame = cur_frame
            cur_frame = next_frame
            
            ret, frame = cap.read()
            if not ret:
                # Если видео закончилось, начинаем сначала
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = cap.read()
                if not ret:
                    break
            
            next_frame = process_frame(frame, scaling_factor)
            frame_count += 1
        
        # Отображение
        cv2.imshow('Motion Detection', motion)
        cv2.imshow('Motion Heatmap', motion_colored)
        
        # Обработка клавиш
        key = cv2.waitKey(30 if not paused else 0)
        
        if key == 27:  # ESC
            break
        elif key == 32:  # SPACE
            paused = not paused
            print("Пауза" if paused else "Возобновление")
    
    print(f"\nОбработано кадров: {frame_count}")
    cap.release()
    cv2.destroyAllWindows()
    print("Программа завершена.")


if __name__ == '__main__':
    main()

СИСТЕМА ОБНАРУЖЕНИЯ ДВИЖЕНИЯ

1. Поиск веб-камеры...
✓ Найдена камера с индексом 0

Источник: Камера

Нажмите ESC для выхода
Нажмите SPACE для паузы


Обработано кадров: 1046
Программа завершена.


![Выход](pictures/1.png)

## Отслеживание объектов с помощью цветовых пространств


In [4]:
import cv2
import numpy as np

def get_frame(cap, scaling_factor):
    """
    Захватывает текущий кадр из веб-камеры и изменяет его размер
    
    Args:
        cap: объект захвата видео
        scaling_factor: коэффициент масштабирования
    
    Returns:
        Измененный кадр
    """
    # Чтение текущего кадра из объекта захвата видео
    ret, frame = cap.read()
    
    if not ret:
        return None
    
    # Изменение размера изображения
    frame = cv2.resize(frame, None, fx=scaling_factor,
                      fy=scaling_factor, interpolation=cv2.INTER_AREA)
    return frame


if __name__ == '__main__':
    # Определение объекта захвата видео
    cap = cv2.VideoCapture(0)
    
    # Проверка успешного открытия камеры
    if not cap.isOpened():
        print("Ошибка: не удалось открыть веб-камеру")
        exit()
    
    # Определение объекта вычитания фона
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    
    # Определим количество предыдущих кадров, которые следует
    # использовать для обучения. Этот фактор управляет скоростью
    # обучения алгоритма. Под скоростью обучения подразумевается
    # скорость, с которой ваша модель будет учиться распознавать
    # фон. Чем выше значение параметра 'history', тем ниже
    # скорость обучения.
    history = 100
    
    # Определение скорости обучения
    learning_rate = 1.0 / history
    
    print("Нажмите ESC для выхода")
    
    # Чтение кадров из веб-камеры до тех пор,
    # пока пользователь не нажмёт клавишу <Esc>
    while True:
        # Захват текущего кадра
        frame = get_frame(cap, 0.5)
        
        if frame is None:
            print("Ошибка: не удалось захватить кадр")
            break
        
        # Вычисление маски
        mask = bg_subtractor.apply(frame, learningRate=learning_rate)
        
        # Преобразование изображения из градаций серого в пространство RGB
        mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        
        # Вывод изображений
        cv2.imshow('Input', frame)
        cv2.imshow('Output', mask_rgb & frame)
        
        # Проверка того, не нажал ли пользователь клавишу <Esc>
        c = cv2.waitKey(10)
        if c == 27:
            break
    
    # Сброс объекта захвата видео
    cap.release()
    
    # Закрытие всех окон
    cv2.destroyAllWindows()

Нажмите ESC для выхода


![Выход](pictures/2.png)

## Отслеживание объектов путём вычитания фоновых изображений


In [5]:
import cv2
import numpy as np


def get_frame(cap, scaling_factor):
    """
    Захватывает текущий кадр из веб-камеры и изменяет его размер
    
    Параметры:
        cap: объект захвата видео
        scaling_factor: коэффициент масштабирования
    
    Возвращает:
        frame: обработанный кадр
    """
    # Чтение текущего кадра из объекта захвата видео
    ret, frame = cap.read()
    
    if not ret:
        return None
    
    # Изменение размера изображения
    frame = cv2.resize(frame, None, fx=scaling_factor,
                      fy=scaling_factor, interpolation=cv2.INTER_AREA)
    
    return frame


if __name__ == '__main__':
    # Определение объекта захвата видео
    cap = cv2.VideoCapture(0)
    
    # Проверка успешного открытия камеры
    if not cap.isOpened():
        print("Ошибка: не удалось открыть веб-камеру")
        exit()
    
    # Определение объекта вычитания фона
    # MOG2 (Mixture of Gaussians) - современный алгоритм вычитания фона
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
    
    # Определение количества предыдущих кадров, которые следует
    # использовать для обучения. Этот фактор управляет скоростью
    # обучения алгоритма. Под скоростью обучения подразумевается
    # скорость, с которой ваша модель будет учиться распознавать
    # фон. Чем выше значение параметра 'history', тем ниже
    # скорость обучения. Вы можете поэкспериментировать с этим
    # значением, чтобы увидеть, как оно влияет на результат.
    history = 100
    
    # Определение скорости обучения
    learning_rate = 1.0 / history
    
    print("Запуск вычитания фона...")
    print("Оставайтесь неподвижными несколько секунд для обучения модели")
    print("Затем начните двигаться, чтобы увидеть эффект")
    print("Нажмите ESC для выхода")
    
    # Чтение кадров из веб-камеры до тех пор,
    # пока пользователь не нажмёт клавишу <Esc>
    while True:
        # Захват текущего кадра
        frame = get_frame(cap, 0.5)
        
        if frame is None:
            print("Ошибка: не удалось получить кадр")
            break
        
        # Вычисление маски с использованием алгоритма вычитания фона
        mask = bg_subtractor.apply(frame, learningRate=learning_rate)
        
        # Преобразование маски из градаций серого в RGB
        # для корректного применения побитового И
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        
        # Применение маски к исходному изображению
        # для выделения движущихся объектов
        output = mask & frame
        
        # Вывод изображений
        cv2.imshow('Input', frame)
        cv2.imshow('Motion mask', mask)
        cv2.imshow('Output', output)
        
        # Проверка того, не нажал ли пользователь клавишу <Esc>
        c = cv2.waitKey(10)
        if c == 27:  # ESC
            break
    
    # Сброс объекта захвата видео
    cap.release()
    
    # Закрытие всех окон
    cv2.destroyAllWindows()
    
    print("Программа завершена")

Запуск вычитания фона...
Оставайтесь неподвижными несколько секунд для обучения модели
Затем начните двигаться, чтобы увидеть эффект
Нажмите ESC для выхода
Программа завершена


![Выход](pictures/3.png)

## Создание интерактивного трекера объектов с помощью алгоритма CAMShift


In [6]:
import cv2
import numpy as np


# Определение класса, содержащего всю функциональность,
# необходимую для отслеживания объектов
class ObjectTracker(object):
    def __init__(self, scaling_factor=0.5):
        # Инициализация объекта захвата видео
        self.cap = cv2.VideoCapture(0)
        
        # Проверка успешного открытия камеры
        if not self.cap.isOpened():
            print("Ошибка: не удалось открыть веб-камеру")
            exit()
        
        # Захват кадра из веб-камеры
        ret, self.frame = self.cap.read()
        
        if not ret:
            print("Ошибка: не удалось захватить кадр")
            exit()
        
        # Масштабный множитель для захваченного изображения
        self.scaling_factor = scaling_factor
        
        # Изменение размера изображения
        self.frame = cv2.resize(self.frame, None, 
                               fx=self.scaling_factor,
                               fy=self.scaling_factor,
                               interpolation=cv2.INTER_AREA)
        
        # Создание окна для отображения кадра
        cv2.namedWindow('Object Tracker')
        
        # Установка функции обратного вызова, отслеживающей события мыши
        cv2.setMouseCallback('Object Tracker', self.mouse_event)
        
        # Инициализация переменной, связанной с ограниченным
        # прямоугольником выбранной области
        self.selection = None
        
        # Инициализация переменной, связанной с начальной позицией
        self.drag_start = None
        
        # Инициализация переменной, связанной с состоянием отслеживания
        self.tracking_state = 0
    
    # Определение метода для отслеживания событий мыши
    def mouse_event(self, event, x, y, flags, param):
        # Преобразование координат X и Y в 16-битовые целые числа NumPy
        x, y = np.int16([x, y])
        
        # Проверка нажатия кнопки мыши
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.tracking_state = 0
        
        # Проверка того, не начал ли пользователь выделять область
        if self.drag_start:
            if flags & cv2.EVENT_FLAG_LBUTTON:
                # Извлечение размеров кадра
                h, w = self.frame.shape[:2]
                
                # Получение начальной позиции
                xi, yi = self.drag_start
                
                # Получение максимальной и минимальной координаты
                x0, y0 = np.maximum(0, np.minimum([xi, yi], [x, y]))
                x1, y1 = np.minimum([w, h], np.maximum([xi, yi], [x, y]))
                
                # Сброс переменной selection
                self.selection = None
                
                # Завершение выделения прямоугольной области
                if x1 - x0 > 0 and y1 - y0 > 0:
                    self.selection = (x0, y0, x1, y1)
            else:
                # Если выделение завершено, начать отслеживание
                self.drag_start = None
                if self.selection is not None:
                    self.tracking_state = 1
    
    # Метод, начинающий отслеживание объекта
    def start_tracking(self):
        print("Выделите объект для отслеживания мышью")
        print("Нажмите ESC для выхода")
        
        # Итерируем до тех пор, пока пользователь не нажмёт клавишу <Esc>
        while True:
            # Захват кадра из веб-камеры
            ret, self.frame = self.cap.read()
            
            if not ret:
                print("Ошибка: не удалось захватить кадр")
                break
            
            # Изменение размера входного кадра
            self.frame = cv2.resize(self.frame, None,
                                   fx=self.scaling_factor,
                                   fy=self.scaling_factor,
                                   interpolation=cv2.INTER_AREA)
            
            # Создание копии кадра
            vis = self.frame.copy()
            
            # Преобразование кадра в цветное пространство HSV
            hsv = cv2.cvtColor(self.frame, cv2.COLOR_BGR2HSV)
            
            # Создание маски на основании предварительно установленных пороговых значений
            mask = cv2.inRange(hsv, np.array((0., 60., 32.)),
                              np.array((180., 255., 255.)))
            
            # Проверка выделения пользователем области
            if self.selection:
                # Извлечение координат выделенного прямоугольника
                x0, y0, x1, y1 = self.selection
                
                # Извлечение окна отслеживания
                self.track_window = (x0, y0, x1 - x0, y1 - y0)
                
                # Извлечение интересующей нас области
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]
                
                # Вычисление гистограммы интересующей нас области
                # HSV-изображения с использованием маски
                hist = cv2.calcHist([hsv_roi], [0], mask_roi, [16], [0, 180])
                
                # Нормализация и переформирование гистограммы
                cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                
                # Извлечение интересующей нас области из кадра
                vis_roi = vis[y0:y1, x0:x1]
                
                # Вычисление негативного изображения (исключительно в целях отображения)
                cv2.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0
            
            # Проверка того, находится ли система в состоянии "отслеживание"
            if self.tracking_state == 1:
                # Сброс переменной selection
                self.selection = None
                
                # Вычисление проекции гистограммы на просвет
                hsv_backproj = cv2.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                
                # Вычисление результата применения операции побитового И
                # к проекции гистограммы на просвет и маске
                hsv_backproj &= mask
                
                # Определение критерия для прекращения работы трекера
                term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)
                
                # Применение алгоритма CAMShift к 'hsv_backproj'
                track_box, self.track_window = cv2.CamShift(hsv_backproj,
                                                            self.track_window,
                                                            term_crit)
                
                # Вычерчивание эллипса вокруг объекта
                cv2.ellipse(vis, track_box, (0, 255, 0), 2)
            
            # Отображение живого видео
            cv2.imshow('Object Tracker', vis)
            
            # Прекратить, если пользователь нажал клавишу <Esc>
            c = cv2.waitKey(5)
            if c == 27:
                break
        
        # Освобождение ресурсов
        self.cap.release()
        
        # Закрытие всех окон
        cv2.destroyAllWindows()


if __name__ == '__main__':
    # Запуск трекера
    ObjectTracker().start_tracking()

Выделите объект для отслеживания мышью
Нажмите ESC для выхода


![Выход](pictures/4.png)

## Отслеживание объектов с использованием оптических потоков


In [7]:
import cv2
import numpy as np


# Определим функцию для отслеживания объекта
def start_tracking():
    # Инициализация объекта захвата видео
    cap = cv2.VideoCapture(0)
    
    # Проверка успешного открытия камеры
    if not cap.isOpened():
        print("Ошибка: не удалось открыть веб-камеру")
        return
    
    # Определение масштабного множителя для кадров
    scaling_factor = 0.5
    
    # Количество отслеживаемых кадров
    num_frames_to_track = 5
    
    # Шаг пропуска
    num_frames_jump = 2
    
    # Инициализация переменных
    tracking_paths = []
    frame_index = 0
    
    # Определение параметров отслеживания
    tracking_params = dict(
        winSize=(11, 11),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
    )
    
    print("Отслеживание началось. Нажмите ESC для выхода")
    
    # Итерирование до тех пор, пока пользователь не нажмёт клавишу <Esc>
    while True:
        # Захват текущего кадра
        ret, frame = cap.read()
        
        if not ret:
            print("Ошибка: не удалось захватить кадр")
            break
        
        # Изменение размеров кадра
        frame = cv2.resize(frame, None, 
                          fx=scaling_factor,
                          fy=scaling_factor,
                          interpolation=cv2.INTER_AREA)
        
        # Преобразование в градации серого
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Создание копии кадра
        output_img = frame.copy()
        
        # Проверим, превышает ли длина отслеживаемых путей нуль
        if len(tracking_paths) > 0:
            # Получение изображений
            prev_img, current_img = prev_gray, frame_gray
            
            # Организация особых точек
            feature_points_0 = np.float32([tp[-1] for tp in tracking_paths]).reshape(-1, 1, 2)
            
            # Вычисление оптического потока
            feature_points_1, _, _ = cv2.calcOpticalFlowPyrLK(
                prev_img, current_img, feature_points_0,
                None, **tracking_params
            )
            
            # Вычисление обратного оптического потока
            feature_points_0_rev, _, _ = cv2.calcOpticalFlowPyrLK(
                current_img, prev_img, feature_points_1,
                None, **tracking_params
            )
            
            # Вычисление разности между прямым и обратным оптическими потоками
            diff_feature_points = abs(feature_points_0 - feature_points_0_rev).reshape(-1, 2).max(-1)
            
            # Извлечение подходящих точек
            good_points = diff_feature_points < 1
            
            # Инициализация переменной
            new_tracking_paths = []
            
            # Итерации по всем подходящим особым точкам
            for tp, (x, y), good_points_flag in zip(
                tracking_paths,
                feature_points_1.reshape(-1, 2),
                good_points
            ):
                # Продолжение, если флаг не равен true
                if not good_points_flag:
                    continue
                
                # Присоединение координат X и Y и проверка того,
                # не превышает ли длина списка пороговое значение
                tp.append((x, y))
                if len(tp) > num_frames_to_track:
                    del tp[0]
                
                new_tracking_paths.append(tp)
                
                # Вычерчивание окружности вокруг особых точек
                cv2.circle(output_img, (int(x), int(y)), 3, (0, 255, 0), -1)
            
            # Обновление путей отслеживания
            tracking_paths = new_tracking_paths
            
            # Вычерчивание линий
            cv2.polylines(output_img, [np.int32(tp) for tp in tracking_paths],
                         False, (0, 150, 0))
        
        # Вход в блок 'if' после пропуска подходящего количества кадров
        if not frame_index % num_frames_jump:
            # Создание маски и вычерчивание окружностей
            mask = np.zeros_like(frame_gray)
            mask[:] = 255
            
            for x, y in [np.int32(tp[-1]) for tp in tracking_paths]:
                cv2.circle(mask, (x, y), 6, 0, -1)
            
            # Вычисление подходящих признаков для отслеживания
            feature_points = cv2.goodFeaturesToTrack(
                frame_gray,
                mask=mask,
                maxCorners=500,
                qualityLevel=0.3,
                minDistance=7,
                blockSize=7
            )
            
            # Проверка существования особых точек; если они
            # существуют, присоединить их к путям отслеживания
            if feature_points is not None:
                for x, y in np.float32(feature_points).reshape(-1, 2):
                    tracking_paths.append([(x, y)])
        
        # Обновление переменных
        frame_index += 1
        prev_gray = frame_gray
        
        # Отображение результата
        cv2.imshow('Input', output_img)
        
        # Проверка того, не нажал ли пользователь клавишу <Esc>
        c = cv2.waitKey(1)
        if c == 27:
            break
    
    # Освобождение ресурсов
    cap.release()
    
    # Закрытие всех окон
    cv2.destroyAllWindows()


if __name__ == '__main__':
    # Запуск трекера
    start_tracking()

Отслеживание началось. Нажмите ESC для выхода


![Вывод](pictures/5.png)

## Обнаружение и отслеживание лиц


### Использование каскадов Хаара для обнаружения лиц


In [8]:
import cv2
import numpy as np

# Загрузка файла каскада Хаара
face_cascade = cv2.CascadeClassifier(
    'haar_cascade_files/haarcascade_frontalface_default.xml'
)

# Если файл не найден локально, попробуйте использовать встроенный путь OpenCV
if face_cascade.empty():
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )

# Проверка корректности загрузки файла каскада
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

# Инициализируем объект захвата видео
cap = cv2.VideoCapture(0)

# Проверка успешного открытия камеры
if not cap.isOpened():
    print("Ошибка: не удалось открыть веб-камеру")
    exit()

# Определение масштабного множителя
scaling_factor = 0.5

print("Детектор лиц запущен. Нажмите ESC для выхода")

# Итерируем до тех пор, пока пользователь не нажмёт клавишу <Esc>
while True:
    # Захват текущего кадра
    ret, frame = cap.read()
    
    if not ret:
        print("Ошибка: не удалось захватить кадр")
        break
    
    # Изменение размера кадра
    frame = cv2.resize(frame, None,
                      fx=scaling_factor,
                      fy=scaling_factor,
                      interpolation=cv2.INTER_AREA)
    
    # Преобразование в градации серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Выполнение детектора лиц для изображения в градациях серого
    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Вычерчивание прямоугольника вокруг лица
    for (x, y, w, h) in face_rects:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
    
    # Отображение результата
    cv2.imshow('Face detection', frame)
    
    # Проверка того, не нажал ли пользователь клавишу <Esc>
    c = cv2.waitKey(1)
    if c == 27:
        break

# Освобождение объекта захвата видео
cap.release()

# Закрытие всех окон
cv2.destroyAllWindows()

[ERROR:0@3025.198] global persistence.cpp:531 open Can't open file: 'haar_cascade_files/haarcascade_frontalface_default.xml' in read mode


Детектор лиц запущен. Нажмите ESC для выхода


![Вывод](pictures/6.png)

### Отслеживание глаз и определение координат взора


In [9]:
import cv2
import numpy as np

# Загрузка файлов каскадов Хаара для лиц и глаз
face_cascade = cv2.CascadeClassifier(
    'haar_cascade_files/haarcascade_frontalface_default.xml'
)
eye_cascade = cv2.CascadeClassifier(
    'haar_cascade_files/haarcascade_eye.xml'
)

# Если файлы не найдены локально, попробуйте использовать встроенные пути OpenCV
if face_cascade.empty():
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )

if eye_cascade.empty():
    eye_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_eye.xml'
    )

# Проверка корректности загрузки файла каскада лица
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

# Проверка корректности загрузки файла каскада глаз
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')

# Инициализация объекта захвата видео
cap = cv2.VideoCapture(0)

# Проверка успешного открытия камеры
if not cap.isOpened():
    print("Ошибка: не удалось открыть веб-камеру")
    exit()

# Определение масштабного множителя
ds_factor = 0.5

print("Детектор глаз запущен. Нажмите ESC для выхода")

# Итерируем до тех пор, пока пользователь не нажмёт клавишу 'Esc'
while True:
    # Захват текущего кадра
    ret, frame = cap.read()
    
    if not ret:
        print("Ошибка: не удалось захватить кадр")
        break
    
    # Изменение размера кадра
    frame = cv2.resize(frame, None, 
                      fx=ds_factor, 
                      fy=ds_factor,
                      interpolation=cv2.INTER_AREA)
    
    # Преобразование в градации серого
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Выполнение детектора лиц для изображения в градациях серого
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Выполнение детектора глаз для каждого обнаруженного лица
    for (x, y, w, h) in faces:
        # Извлечение интересующей нас области изображения лица
        # в градациях серого
        roi_gray = gray[y:y+h, x:x+w]
        
        # Извлечение интересующей нас области цветного изображения лица
        roi_color = frame[y:y+h, x:x+w]
        
        # Выполнение детектора глаз в интересующей нас области
        # изображения в градациях серого
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # Вычерчивание окружностей вокруг глаз
        for (x_eye, y_eye, w_eye, h_eye) in eyes:
            center = (int(x_eye + 0.5 * w_eye), int(y_eye + 0.5 * h_eye))
            radius = int(0.3 * (w_eye + h_eye))
            color = (0, 255, 0)
            thickness = 3
            cv2.circle(roi_color, center, radius, color, thickness)
    
    # Отобразить вывод
    cv2.imshow('Eyes detection', frame)
    
    # Проверка того, не нажал ли пользователь клавишу <Esc>
    c = cv2.waitKey(1)
    if c == 27:
        break

# Освобождение объекта захвата видео
cap.release()

# Закрытие всех окон
cv2.destroyAllWindows()

[ERROR:0@3094.480] global persistence.cpp:531 open Can't open file: 'haar_cascade_files/haarcascade_frontalface_default.xml' in read mode
[ERROR:0@3094.480] global persistence.cpp:531 open Can't open file: 'haar_cascade_files/haarcascade_eye.xml' in read mode


Детектор глаз запущен. Нажмите ESC для выхода


![Вывод](pictures/7.png)